## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Изучение общей информации

In [1]:
import pandas as pd  #вызов библиотеки pandas
data.info()   #вывод общей информации о таблице с данными

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Во входной таблице данных 12 столбцов и 21525 строк, одна строка - данные о платежеспособности одного клиента. В столбцах общий трудовой стаж и ежемесячный доход есть пропущенные значения. По типам данных смущает только 'float64' для стажа в днях (не вижу смысла хранить не целые дни). Посмотрю на несколько строк таблицы подробнее и сделаю еще выводы. Также выведу названия столбцов и оценю, все ли корректно.

In [2]:
data.columns  #вывод списка названий столбцов

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Столбцы названы корректно, изменения вносить не требуется.

In [3]:
data.head(15) #вывод первых 15 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data['children'].unique() #уникальные значения столбца с кол-вом детей

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [5]:
data['days_employed'].min() #минимальный стаж

-18388.949900568383

In [6]:
data['days_employed'].max() #максимальный стаж

401755.40047533

In [7]:
data['dob_years'].min() #минимальный возраст

0

In [8]:
data['dob_years'].max() #максимальный возраст

75

In [9]:
data['education'].unique() #уникальные значения столбца с уровнем образования

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [10]:
data['education_id'].unique() #уникальные значения столбца ид уровня образования

array([0, 1, 2, 3, 4])

In [11]:
data['family_status'].unique() #уникальные значения столбца с семейным положением

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [12]:
data['family_status_id'].unique() #уникальные значения столбца ид семейного положения

array([0, 1, 2, 3, 4])

In [13]:
data['gender'].unique() #уникальные значения столбца с полом

array(['F', 'M', 'XNA'], dtype=object)

In [14]:
data['income_type'].unique() #уникальные значения столбца тип занятости

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [15]:
data['total_income'].min() #минимальный заработок

20667.26379327158

In [16]:
data['total_income'].max() #максимальный заработок

2265604.028722744

In [17]:
data['debt'].unique() #уникальные значения столбца была ли задолженность

array([0, 1])

### Вывод

Итак, замеченные проблемы, которые нужно решить:
1. Обработать пропуски в доходах и стажах. Возможно эти пропуски связаны с тем, что эти люди никогда не работали
2. Сделать замену нулевого возраста.
3. Привести к одному регистру значения столбца - уровень образования и семейное положение. Думаю, что эта ошибка допущена из-за свободного ввода данных, идеально сделать там выбор из ограниченного списка.
4. Неопределенное значение в столбце с полом.
5. Обработать значения с кол-вом детей (странные значения -1 и 20, вероятно допущена ошибка при вводе 1 и 2).
6. Значения столбца с целью кредита нужно лемматизировать для дальнейшей категоризации.
7. Самый интересный и в то же время странный столбец  - стаж в днях (какие-то отрицательные, не целые значения). Идеально было бы уточнить их происхождение у специалиста, предоставившего данные. Но я предположу, что стаж вычислялся как разница двух дат, при этом была какая-то базовая дата и периоды до этой даты стали отрицательными. Также есть аномально высокие значения (максимальное значение стажа в столбце 401755 дней - это больше 1000 лет).
8. Значения данных стажа и дохода перевести в целочисленный тип.

### Шаг 2. Предобработка данных

1. Обработка пропусков

1.1 Обработка пропусков в доходах

In [18]:
data.isnull().sum()      #суммарное количество пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Строк с пропусками минимум 2174, это 10,1% от общего кол-ва, слишком много, удалять нельзя. Интересно, что количество пропусков в столбцах со стажем и доходом одинаковое. Предположу, что это одни и те же строки. И если это так, то возможно, что у людей, которые никогда не работали, нет стажа и нет дохода. Проверю своё предположение следующим образом - удалю все строки с пустыми значениями и, если останется всего строк 21525-2174=19351, то предположение верно.

In [19]:
data_new = data.dropna()  #удаляю строки с пропусками и смотрю инфо

data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


Предположение подтвердилось, тем интереснее. Нужно изучить эти строки подробнее, чтобы понять, чем их заполнить.

In [20]:
data[data['days_employed'] == 0].count() #проверяю, есть ли нулевые значения в столбце стаж


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [21]:
data[data['total_income'] == 0].count()#проверяю, есть ли нулевые значения в столбце доход

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Нулевых значений нет, поэтому смело заполню пропуски нулями и продолжу анализ по ним.

In [22]:
data_new = data.fillna(0) #заменяю пропущенные значения нулями, сохраняю в новую таблицу
data_new.info()  #проверяю, все ли заменилось

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Все пропуски заменены на нули. Теперь проверю предположение о том, что эти люди безработные, посмотрю какой тип занятости у них.

In [23]:
data_new[data_new['days_employed'] == 0].groupby('income_type').count() #сгруппирую строки с нулевыми стажами по типу занятости,
                                                                        #выведу кол-во по каждому типу занятости

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,147,147,147,147,147,147,147,147,147,147,147
компаньон,508,508,508,508,508,508,508,508,508,508,508
пенсионер,413,413,413,413,413,413,413,413,413,413,413
предприниматель,1,1,1,1,1,1,1,1,1,1,1
сотрудник,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105


Нет ни одного безработного, значит заполнение нулями не подойдет, логичнее заменить эти значения медианой дохода по типу занятости. Посчитаю медианы дохода по типам занятости, при этом строки с нулями в расчет брать не буду.

In [24]:
# медианные значения заработков по типам занятости (без учета нулевых значений)
income_type_median = data_new[data_new['days_employed'] != 0].groupby('income_type')['total_income'].median()
income_type_median

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Сделаю замену на медианные значения по типу дохода.

In [25]:
data_new.loc[data_new['income_type'] == 'госслужащий', 'total_income'] = data_new['total_income'].replace(0, income_type_median[2])
data_new.loc[data_new['income_type'] == 'компаньон', 'total_income'] = data_new['total_income'].replace(0, income_type_median[3])
data_new.loc[data_new['income_type'] == 'сотрудник', 'total_income'] = data_new['total_income'].replace(0, income_type_median[6])
data_new.loc[data_new['income_type'] == 'предприниматель', 'total_income'] = data_new['total_income'].replace(0, income_type_median[5])
data_new.loc[data_new['income_type'] == 'пенсионер', 'total_income'] = data_new['total_income'].replace(0, income_type_median[4])

data_new[data_new['total_income'] == 0].groupby('income_type').count() # проверю, не осталось ли в доходе нулей


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,


Замена произведена успешно.

1.2 Обработка нулевых возрастов

In [26]:
data_new[data_new['dob_years'] == 0].count() #кол-во строк с нулевым возрастом

children            101
days_employed       101
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income        101
purpose             101
dtype: int64

Посмотрю к каким типам занятости они относятся.

In [27]:
data_new[data_new['dob_years'] == 0].groupby('income_type')['income_type'].count()

income_type
госслужащий     6
компаньон      20
пенсионер      20
сотрудник      55
Name: income_type, dtype: int64

Найду медианные значения возраста по типам занятости

In [28]:
dob_years_median = data_new[data_new['dob_years'] != 0].groupby('income_type')['dob_years'].median()
dob_years_median

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

Сделаю замену нулевых возрастов на медианные значения возраста по типу занятости

In [29]:
data_new.loc[data_new['income_type'] == 'госслужащий', 'dob_years'] = data_new['dob_years'].replace(0, dob_years_median[2])
data_new.loc[data_new['income_type'] == 'компаньон', 'dob_years'] = data_new['dob_years'].replace(0, dob_years_median[3])
data_new.loc[data_new['income_type'] == 'пенсионер', 'dob_years'] = data_new['dob_years'].replace(0, dob_years_median[4])
data_new.loc[data_new['income_type'] == 'сотрудник', 'dob_years'] = data_new['dob_years'].replace(0, dob_years_median[6])

data_new[data_new['dob_years'] == 0].count() # проверю, все ли нули заменились

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Замена произведена успешно.

1.3 Обработка пропусков в стаже

Пропуски в стаже заменю на медианные значения по возрастным категориям. Для этого выделю следующие возрастные категории: до 30 лет, от 31 до 40, от 41 до 55 и больше 55. И напишу функцию, которая будет по входному аргументу возраст определять возрастную категорию.

In [30]:
dob_years_groups = [30, 40, 55]

def dob_years_category(dob_years):
    if dob_years <= dob_years_groups[0]:
        return 'до 30'
    elif dob_years <= dob_years_groups[1]:
        return 'от 31 до 40'
    elif dob_years <= dob_years_groups[2]:
        return 'от 41 до 55'
    else:
        return 'больше 55'

#методом apply() и написанной функцией добавлю столбец с возрастной категорией 'dob_years_category'
data_new['dob_years_category'] = data_new['dob_years'].apply(dob_years_category) 
data_new.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,от 41 до 55
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,от 31 до 40
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,от 31 до 40
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,от 31 до 40
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,от 41 до 55


Теперь можно считать медианные значения стажа по возрастным группам и делать замену пропусков в стаже на эти значения. Для подсчета исключу нулевые стажи (это те, которые были пропусками изначально).

In [31]:
days_employed_median = data_new[data_new['days_employed'] != 0].groupby('dob_years_category')['days_employed'].median()
days_employed_median

dob_years_category
больше 55      347618.524632
до 30           -1041.699372
от 31 до 40     -1600.879058
от 41 до 55     -1757.344692
Name: days_employed, dtype: float64

Нужные значения получены, можно делать замену

In [32]:
data_new.loc[data_new['dob_years_category'] == 'больше 55', 'days_employed'] = data_new['days_employed'].replace(0, days_employed_median[0])
data_new.loc[data_new['dob_years_category'] == 'до 30', 'days_employed'] = data_new['days_employed'].replace(0, days_employed_median[1])
data_new.loc[data_new['dob_years_category'] == 'от 31 до 40', 'days_employed'] = data_new['days_employed'].replace(0, days_employed_median[2])
data_new.loc[data_new['dob_years_category'] == 'от 41 до 55', 'days_employed'] = data_new['days_employed'].replace(0, days_employed_median[3])

data_new[data_new['days_employed'] == 0].count() # проверю, все ли нули заменились

children              0
days_employed         0
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
dob_years_category    0
dtype: int64

### Вывод

Обнаружены пропуски в столбцах общий стаж и месячный доход, причем в одних и тех же строках. Кол-во таких строк - 2174, это 10,1% от общего кол-ва, слишком много, удалять было нельзя. Заменять нулями тоже некорректно, так как это сильно бы занизило показатель дохода в типах занятости. В итоге пропуски в доходах заменены на медианные значения по соответствующим типам занятости. Нулевые значения возраста заменены на медианные значения по типу занятости, а пропуски в стаже заменены на медианные значения по возрастным группам.

2. Замена типа данных

2.1 Перевод вещественного типа в целочисленный значений столбцов доход и стаж

In [33]:
data_new['total_income'] = data_new['total_income'].astype('int')
data_new['days_employed'] = data_new['days_employed'].astype('int')
data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children              21525 non-null int64
days_employed         21525 non-null int64
dob_years             21525 non-null int64
education             21525 non-null object
education_id          21525 non-null int64
family_status         21525 non-null object
family_status_id      21525 non-null int64
gender                21525 non-null object
income_type           21525 non-null object
debt                  21525 non-null int64
total_income          21525 non-null int64
purpose               21525 non-null object
dob_years_category    21525 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


### Вывод

Я использовала метод astype(), так как пропущенные значения уже отсутствуют, иначе нужно было бы применять to_numeric() с параметром errors для встреч с некорректными значениями.

3. Обработка дубликатов

3.1 Перевод значений столбцов образование и семейное положение к нижнему регистру

In [34]:
data_new['education'] = data_new['education'].str.lower() #привожу значения столбца образование к нижнему регистру
data_new['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [35]:
data_new['family_status'] = data_new['family_status'].str.lower() #привожу значения столбца семейное положение к нижнему регистру
data_new['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

3.2 Замена ошибок в количестве детей (значения -1 и 20)

In [36]:
data_new['children'] = data_new['children'].replace(-1, 1) # -1 на 1
data_new['children'] = data_new['children'].replace(20, 2) # 20 на 2
data_new['children'].unique()

array([1, 0, 3, 2, 4, 5])

3.3 Анализ значения 'XNA' в столбце с полом

In [37]:
data_new[data_new['gender'] == 'XNA'].count() #кол-во 'XNA' в столбце с полом

children              1
days_employed         1
dob_years             1
education             1
education_id          1
family_status         1
family_status_id      1
gender                1
income_type           1
debt                  1
total_income          1
purpose               1
dob_years_category    1
dtype: int64

Найдена всего 1 строка с неопределенным полом, но к сожалению по тем данным, которые имеются, установить пол не удастся. Оставлю как есть.

3.4 Удаление дубликатов

In [38]:
data_new.duplicated().sum() # считаю количество дубликатов

71

In [39]:
data_new = data_new.drop_duplicates().reset_index(drop = True) #удаляю дубликаты
data_new.duplicated().sum() # проверяю кол-во после удаления

0

### Вывод

Основные ошибки, которые были исправлены - введенные вручную виды образования. Писала в самом начале о том, что эту ошибку легко избежать, если ограничить выбор значений списком. Для удаления дубликатов использовала метод drop_duplicates(), так как он ищет идентичные строки. Возникновение дубликатов может быть вызвано как технической ошибкой, так и человеческим фактором.

4. Лемматизация

In [40]:
from pymystem3 import Mystem # импортирую библиотеку pymystem3 и collections
from collections import Counter
m = Mystem()

data_new['purpose'].value_counts() # посчитаю, сколько всего вариантов целей кредита

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Список впечатляет) Выделю ключевые слова - 'свадьба', 'недвижимость', 'коммерческий', 'сдача', 'жилье', 'автомобиль' и 'образование'.

In [41]:
purpose_category = ['свадьба', 'коммерческий', 'недвижимость', 'сдача', 'жилье', 'автомобиль', 'образование']

#проведу лемматизацию и заменю полученный список лемм на ключевые слова из моего списка категорий
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in purpose_category:
        if word in lemma:
            lemma = word
    return lemma

data_new['purpose_category'] = data['purpose'].apply(lemmatize)        
data_new.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 41 до 55,жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 31 до 40,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 31 до 40,жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 31 до 40,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,от 41 до 55,свадьба


Посмотрю, как распределились строки по категориям

In [42]:
data_new['purpose_category'].value_counts()

недвижимость    5038
автомобиль      4295
образование     4011
жилье           3810
свадьба         2340
коммерческий    1310
сдача            650
Name: purpose_category, dtype: int64

Объединю 'недвижимость' и 'жилье' в 'недвижимость', так как жилье это недвижимость. И объединю 'коммерческий' и 'сдача' в 'коммерческий'.

In [43]:
data_new.loc[data_new['purpose_category'] == 'жилье', 'purpose_category'] = 'недвижимость'
data_new.loc[data_new['purpose_category'] == 'сдача', 'purpose_category'] = 'коммерческий'
data_new['purpose_category'].value_counts()

недвижимость    8848
автомобиль      4295
образование     4011
свадьба         2340
коммерческий    1960
Name: purpose_category, dtype: int64

### Вывод

По каждому клиенту из цели получения кредита выделила одно ключевое слово. Данная процедура поможет проводить сравнение по категориям, при этом цель получения кредита каждого клиента сохраняет свою смысловую нагрузку.

### Категоризация данных

Ранее я провела категоризацию по возрасту и цели получения кредита. Осталось провести категоризацию по доходам, чтобы можно было отвечать на поставленные вопросы. Воспользуюсь методом describe(), оценю разброс значений в доходах и возьму за граничные значения категорий квантили 25%, 50%, 75%. Создам функцию, которая по входному аргументу доход будет определять категорию

In [44]:
data_new['total_income'].describe() 

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [45]:
total_income_groups = [107623, 142594, 195820]

def total_income_category(total_income):
    if total_income <= total_income_groups[0]:
        return 'доход ниже среднего'
    elif total_income <= total_income_groups[1]:
        return 'средний доход'
    elif total_income <= total_income_groups[2]:
        return 'доход вышего среднего'
    else:
        return 'высокий доход'
    
#методом apply() и написанной функцией добавлю столбец с категорией дохода 'dob_years_category'
data_new['total_income_category'] = data_new['total_income'].apply(total_income_category) 
data_new.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category,purpose_category,total_income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 41 до 55,недвижимость,высокий доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 31 до 40,автомобиль,средний доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 31 до 40,недвижимость,доход вышего среднего
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 31 до 40,образование,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,от 41 до 55,свадьба,доход вышего среднего


In [46]:
data_new['total_income_category'].value_counts()

средний доход            5479
доход ниже среднего      5364
высокий доход            5364
доход вышего среднего    5247
Name: total_income_category, dtype: int64

In [47]:
data_new['dob_years_category'].value_counts()

от 41 до 55    7568
от 31 до 40    5813
больше 55      4356
до 30          3717
Name: dob_years_category, dtype: int64

### Вывод

Данные по доходу распределила по 4-ем категориям, в каждой около 25% имеющихся для анализа клиентов.
Данные по возрасту категоризированы в процессе обработки пропусков в стаже и распределены по 4-ем возрастным группам. Наибольшее количество в группе от 41 до 55 лет - 7568 клиентов, что составляет 35,1% от общего числа данных, эта категория людей чаще всего обращается за получением кредита. Наименьшее кол-во в группе до 30 лет - 3717 клиентов - 14,8% от общего числа, реже остальных берут кредит.
Данные по цели кредита были категоризированы при лемматизации, где выявлено, что 41% обращений связан с целью получения средств на операции с жилой недвижимостью и 9% обращений связано с операциями по коммерческой недвижимости.

### Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построю сводную таблицу, сгруппировав данные по столбцу с кол-вом детей и выводя в значениях суммарное кол-во клиентов, имеющих просрочку.

In [48]:
data_new_pivot = data_new.pivot_table(index = ['children'], values = 'debt', aggfunc = 'sum')
data_new_pivot

,debt
children,
0,1063
1,445
2,202
3,27
4,4
5,0


In [49]:
data_new['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [50]:
#создам функцию для категоризации клиентов - '1'-есть дети, '0'-нет детей
def children_category(children):
    if children > 0:
        return 1
    else:
        return 0
#добавлю столбец с категорией по детям
data_new['children_category'] = data_new['children'].apply(children_category)

In [51]:
data_new_pivot = data_new.pivot_table(index = ['children_category'], values = 'debt').round(4)
data_new_pivot

,debt
children_category,
0,0.0754
1,0.0921


### Вывод

Итак, всего клиентов не имеющих детей - 14091, из них имеют задолженность 1063, то есть вероятность просрочки у клиентов без детей 7,5%. Всего клиентов, имеющих детей - 7434, из них имеют задолженность 678, вероятность просрочки 9,2%. Это говорит о том, что клиенты, имеющие детей, чаще допускают просрочки по платежам, чем те, у кого детей нет. Вероятно это связано с тем, что при трудном материальном положении клиент выбирает задолжать банку, максимально сохраняя обеспечение детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построю сводную таблицу, сгруппировав данные по столбцу с семейным положением и выводя в значениях суммарное кол-во клиентов, имеющих просрочку.

In [52]:
data_new['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [53]:
data_new_pivot = data_new.pivot_table(index = ['family_status'], values = 'debt').round(4)
data_new_pivot

,debt
family_status,
в разводе,0.0711
вдовец / вдова,0.0657
гражданский брак,0.0935
женат / замужем,0.0755
не женат / не замужем,0.0975


### Вывод

Получились следующие вероятности просрочки в зависимости от семейного положения: 
вдовец / вдова - 6,5%
в разводе - 7,1%
женат / замужем - 7,5%
гражданский брак - 9,3%
не женат / не замужем - 9,7%

То есть люди, которые в официальном браке или которые когда-то в нем были (вдовец, развод) реже допускают просрочки по кредитным платежам. Думаю, они более ответственны, имея опыт семейной жизни.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построю сводную таблицу, сгруппировав данные по столбцу с категорией дохода и выводя в значениях суммарное кол-во клиентов, имеющих просрочку.

In [54]:
data_new['total_income_category'].value_counts()

средний доход            5479
доход ниже среднего      5364
высокий доход            5364
доход вышего среднего    5247
Name: total_income_category, dtype: int64

In [55]:
data_new_pivot = data_new.pivot_table(index = ['total_income_category'], values = 'debt', aggfunc = 'sum')
data_new_pivot

,debt
total_income_category,
высокий доход,383
доход вышего среднего,448
доход ниже среднего,427
средний доход,483


In [56]:
data_new_pivot = data_new.pivot_table(index = ['total_income_category'], values = 'debt').round(4)
data_new_pivot

,debt
total_income_category,
высокий доход,0.0714
доход вышего среднего,0.0854
доход ниже среднего,0.0796
средний доход,0.0882


### Вывод

Получились следующие вероятности просрочки в зависимости от дохода: 
высокий доход - 7,1%
доход вышего среднего - 8,5%
доход ниже среднего - 8%
средний доход - 8,8%

Интересно, но наибольшая вероятность задолжать у клиентов со средним доходом против клиентов с доходом ниже среднего. Вероятно это потому, что люди с меньшим доходом более внимательны к тратам и лучше себя контролируют. Самый платежеспособный клиент - клиент с высоким доходом, что и логично.

- Как разные цели кредита влияют на его возврат в срок?

Построю сводную таблицу, сгруппировав данные по столбцу с целью кредита и выводя в значениях суммарное кол-во клиентов, имеющих просрочку.

In [57]:
data_new['purpose_category'].value_counts()

недвижимость    8848
автомобиль      4295
образование     4011
свадьба         2340
коммерческий    1960
Name: purpose_category, dtype: int64

In [58]:
data_new_pivot = data_new.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = 'sum')
data_new_pivot

,debt
purpose_category,
автомобиль,345
коммерческий,173
недвижимость,712
образование,333
свадьба,178


In [59]:
data_new_pivot = data_new.pivot_table(index = ['purpose_category'], values = 'debt').round(4)
data_new_pivot

,debt
purpose_category,
автомобиль,0.0803
коммерческий,0.0883
недвижимость,0.0805
образование,0.0830
свадьба,0.0761


### Вывод

Получились следующие вероятности просрочки в зависимости от цели получения кредита: 
недвижимость - 8%
автомобиль - 8%
образование - 8,3%
свадьба - 7,6%
коммерческий - 8,8%

Менее платежеспособные оказались люди, берущие кредит на коммерческие цели (покупка квартиры для сдачи и т.п.). Думаю, что это связано с тем, что люди сразу рассчитывают получать доход с купленной недвижимости, но им это не всегда удается, а из текущего дохода не запланировано делать платежи по этому кредиту. Наиболее платежеспособны оказались клиенты, берущие кредит на проведение свадьбы, так как в браке бюджет чаще объединяется и платежеспособность увеличивается.

### Шаг 4. Общий вывод

"Хороший" клиент - это клиент, находящийся когда-то в браке, не имеющий детей, с высоким доходом и берущий кредит на проведение свадьбы.